In [1]:
pip install pyodbc

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import pyodbc
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-LJ2060M\SQLEXPRESS;"
    "DATABASE=movies;"
    "Trusted_Connection=yes"
)

cursor = conn.cursor()
cursor.execute("SELECT * FROM movies")

for row in cursor.fetchall():
    print(row)

conn.close()

<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Admin\AppData\Local\Temp\ipykernel_12672\3125374181.py:4: SyntaxWarning: invalid escape sequence '\S'
  "SERVER=DESKTOP-LJ2060M\SQLEXPRESS;"


(1001, 'Avatar', 'Thriller', datetime.date(2023, 2, 11), 'Director A', 2001, 'German', 'USA', Decimal('292417794.00'), Decimal('1973288843.00'), 97, Decimal('8.6'), 96, 'Studio A', 'Pre-Production', 'PG-13', 'Yes', datetime.datetime(2026, 1, 16, 23, 13, 23, 120000))
(1002, 'Inception', 'Action', datetime.date(2017, 3, 15), 'Director B', 2014, 'Spanish', 'USA', Decimal('80253577.00'), Decimal('2425020913.00'), 163, Decimal('8.4'), 55, 'Studio B', 'Post-Production', 'PG-13', 'No', datetime.datetime(2026, 1, 16, 23, 13, 23, 120000))
(1003, 'The Dark Knight', 'Drama', datetime.date(2001, 7, 26), 'Director C', 2018, 'Spanish', 'USA', Decimal('188814137.00'), Decimal('2265213337.00'), 114, Decimal('8.8'), 59, 'Studio C', 'Pre-Production', 'G', 'No', datetime.datetime(2026, 1, 16, 23, 13, 23, 120000))
(1004, 'Interstellar', 'Romance', datetime.date(2019, 1, 13), 'Director D', 1994, 'German', 'Germany', Decimal('235147201.00'), Decimal('482239058.00'), 163, Decimal('8.8'), 63, 'Studio D', 'Pos

In [4]:
#FINAL CODE
import pyodbc
from datetime import datetime
import os

class RestaurantBilling:

    def __init__(self):
        try:
            self.conn = pyodbc.connect(
                "Driver={ODBC Driver 17 for SQL Server};"
                "Server=DESKTOP-LJ2060M\SQLEXPRESS;"
                "Database=bill_printing;"
                "Trusted_Connection=yes;"
            )
            self.cursor = self.conn.cursor()
            print("Connection Successful \n")
        except Exception as e:
            print("Connection Failed ", e)

    # ---------------------------------
    # Utility: Integer Validation
    # ---------------------------------
    def get_valid_integer(self, message):
        while True:
            value = input(message).strip()
            if value.isdigit():
                return int(value)
            else:
                print("Only numbers allowed Please try again.\n")

    # ---------------------------------
    # Utility: Yes/No Validation
    # ---------------------------------
    def get_yes_no(self, message):
        while True:
            choice = input(message).strip().lower()

            if choice == 'y':
                return 'y'
            elif choice == 'n':
                return 'n'
            else:
                print("Please enter only 'y' or 'n' \n")

    # ---------------------------------
    # Show Menu
    # ---------------------------------
    def show_menu(self):
        print("\n----------- MENU -----------")
        self.cursor.execute("SELECT * FROM menu")
        rows = self.cursor.fetchall()

        for row in rows:
            print(f"{row[0]}. {row[1].strip():15} ₹{row[2]}")
        print("-----------------------------\n")

    # ---------------------------------
    # Take Order
    # ---------------------------------
    def take_order(self):

        print("\nStart Taking Order (Enter 0 to Stop)\n")

        while True:
            item_id = self.get_valid_integer("Enter Item Number: ")

            if item_id == 0:
                print("Order Taking Stopped \n")
                break

            quantity = self.get_valid_integer("Enter Quantity: ")

            # Check if item exists
            self.cursor.execute(
                "SELECT items FROM menu WHERE item_id = ?",
                (item_id,)
            )
            item = self.cursor.fetchone()

            if not item:
                print("Invalid Item ID \n")
                continue

            item_name = item[0].strip()

            # Insert order with bill_id
            self.cursor.execute(
                "INSERT INTO orders (bill_id, item_id, quantity, order_date) VALUES (?, ?, ?, ?)",
                (self.bill_id, item_id, quantity, datetime.now())
            )
            self.conn.commit()

            print(f"Added: {item_name} x{quantity} \n")

    # ---------------------------------
    # Generate Bill
    # ---------------------------------
    def generate_bill(self):

        query = """
        SELECT 
            m.items,
            SUM(o.quantity) AS total_qty,
            m.price,
            SUM(o.quantity * m.price) AS total_price
        FROM orders o
        JOIN menu m ON o.item_id = m.item_id
        WHERE o.bill_id = ?
        GROUP BY m.items, m.price
        """

        self.cursor.execute(query, (self.bill_id,))
        rows = self.cursor.fetchall()

        if not rows:
            print("No orders found \n")
            return 0

        now = datetime.now()
        total_amount = 0
        sr = 1

        bill_text = ""
        bill_text += "_"*45 + "\n"
        bill_text += "       Welcome to Hotel Chavishta\n"
        bill_text += "_"*45 + "\n"
        bill_text += f"Bill ID: {self.bill_id}\n"
        bill_text += f"Date: {now.strftime('%d/%m/%Y')}  Time: {now.strftime('%H:%M:%S')}\n"
        bill_text += "_"*45 + "\n"
        bill_text += f"{'Sr':<4}{'Item':<15}{'Qty':<6}{'Total'}\n"
        bill_text += "_"*45 + "\n"

        for item, qty, price, total in rows:
            bill_text += f"{sr:<4}{item.strip():<15}{qty:<6}₹{total}\n"
            total_amount += total
            sr += 1

        bill_text += "_"*45 + "\n"
        bill_text += f"{'Total Amount':<25}₹{total_amount}\n"
        bill_text += "_"*45 + "\n"

        # 🔹 Show on screen
        print(bill_text)

        # 🔹 Store in object for later saving
        self.bill_text = bill_text

        return total_amount

    # ---------------------------------
    # Print Bill
    # ---------------------------------
    def print_bill(self):

        choice = self.get_yes_no("\nPrint Bill? (y/n): ")

        if choice == 'y':

            import os

            folder_name = "Bills"

            if not os.path.exists(folder_name):
                os.makedirs(folder_name)

            file_path = os.path.join(folder_name, f"Bill_{self.bill_id}.txt")

            with open(file_path, "w", encoding="utf-8") as f:
                f.write(self.bill_text)

            print(f"Bill Saved Successfully in {file_path}\n")

        else:
            print("Bill Not Printed.\n")

    # ---------------------------------
    # Main Run Method
    # ---------------------------------
    def run(self):

        print("\nWelcome to Hotel Chavishta\n")

        # Generate unique bill_id using timestamp
        self.bill_id = datetime.now().strftime("%Y%m%d%H%M%S")

        self.show_menu()

        self.take_order()

        delivered = self.get_yes_no("Is Order Delivered? (y/n): ")

        if delivered != 'y':
            print("Order not delivered. Cannot generate bill.")
            return

        print("Order Delivered \n")

        gen_bill = self.get_yes_no("Generate Bill? (y/n): ")

        if gen_bill == 'y':
            total = self.generate_bill()

            if total > 0:
                self.print_bill()
        else:
            print("Bill generation cancelled.")


# ---------------------------------
# Run Program
# ---------------------------------
system = RestaurantBilling()
system.run()

<>:12: SyntaxWarning: invalid escape sequence '\S'
<>:12: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14804\2419002929.py:12: SyntaxWarning: invalid escape sequence '\S'
  "Server=DESKTOP-LJ2060M\SQLEXPRESS;"


Connection Successful 


Welcome to Hotel Chavishta


----------- MENU -----------
1. coffee          ₹50
2. chai            ₹20
3. cold coffee     ₹70
4. fries           ₹100
5. pizza           ₹150
6. burger          ₹90
7. pasta           ₹180
8. momos           ₹80
9. waffle          ₹160
10. ice cream       ₹70
-----------------------------


Start Taking Order (Enter 0 to Stop)

Added: coffee x1 

Added: fries x1 

Added: pizza x1 

Added: waffle x1 

Order Taking Stopped 

Order Delivered 

_____________________________________________
       Welcome to Hotel Chavishta
_____________________________________________
Bill ID: 20260217225824
Date: 17/02/2026  Time: 22:58:57
_____________________________________________
Sr  Item           Qty   Total
_____________________________________________
1   coffee         1     ₹50
2   fries          1     ₹100
3   pizza          1     ₹150
4   waffle         1     ₹160
_____________________________________________
Total Amount             ₹

In [9]:
import pyodbc
from datetime import datetime
import os

class RestaurantBilling:

    def __init__(self):
        try:
            self.conn = pyodbc.connect(
                "Driver={ODBC Driver 17 for SQL Server};"
                "Server=DESKTOP-LJ2060M\SQLEXPRESS;"
                "Database=bill_printing;"
                "Trusted_Connection=yes;"
            )
            self.cursor = self.conn.cursor()
            print("Database Connected Successfully\n")
        except:
            print("Connection Failed")

    # ----------------------------
    # Show Menu
    # ----------------------------
    def show_menu(self):
        print("\n------ MENU ------")
        self.cursor.execute("SELECT * FROM menu")
        rows = self.cursor.fetchall()

        for row in rows:
            print(row[0], row[1].strip(), "₹", row[2])
        print("------------------\n")

    # ----------------------------
    # Take Order
    # ----------------------------
    def take_order(self):
        print("Start Taking Order (Enter 0 to Stop)\n")

        while True:
            item_id = int(input("Enter Item Number: "))

            if item_id == 0:
                print("Order Taking Stopped\n")
                break

            quantity = int(input("Enter Quantity: "))

            # Check item exists
            self.cursor.execute("SELECT items FROM menu WHERE item_id=?", (item_id,))
            item = self.cursor.fetchone()

            if item is None:
                print("Invalid Item ID\n")
                continue

            item_name = item[0].strip()

            self.cursor.execute(
                "INSERT INTO orders (bill_id, item_id, quantity, order_date) VALUES (?, ?, ?, ?)",
                (self.bill_id, item_id, quantity, datetime.now())
            )
            self.conn.commit()

            print("Added:", item_name, "x", quantity, "\n")

    # ----------------------------
    # Generate Bill
    # ----------------------------
    def generate_bill(self):

        self.cursor.execute("""
            SELECT m.items, o.quantity, m.price
            FROM orders o
            JOIN menu m ON o.item_id = m.item_id
            WHERE o.bill_id = ?
        """, (self.bill_id,))

        rows = self.cursor.fetchall()

        if not rows:
            print("No orders found\n")
            return 0

        bill_text = ""

        line = "-" * 50
        bill_text += line + "\n"
        bill_text += "|{:^48}|\n".format("Welcome Hotel Chavishta")
        bill_text += line + "\n"
        bill_text += "| {:<3} {:<15} {:<8} {:>8} |\n".format("sr", "Menu", "qunt", "price")
        bill_text += line + "\n"

        total_amount = 0
        sr = 1

        for row in rows:
            item = row[0].strip()
            qty = row[1]
            price = row[2]
            total = qty * price

            bill_text += "| {:<3} {:<15} {:<8} {:>8} |\n".format(sr, item, qty, total)

            total_amount += total
            sr += 1

        bill_text += line + "\n"
        bill_text += "| {:<18} {:>18} |\n".format("Total", total_amount)
        bill_text += line + "\n"

        print(bill_text)

        self.bill_text = bill_text
        self.total_amount = total_amount

        return total_amount

    # ----------------------------
    # Save Bill
    # ----------------------------
    def save_bill(self):

        choice = input("Print Bill? (y/n): ").lower()

        if choice == 'y':

            if not os.path.exists("Bills"):
                os.makedirs("Bills")

            file_path = f"Bills/Bill_{self.bill_id}.txt"

            with open(file_path, "w", encoding="utf-8") as f:
                f.write(self.bill_text)

            print("Bill Saved Successfully\n")

        else:
            print("Bill Not Printed\n")

    # ----------------------------
    # Run
    # ----------------------------
    def run(self):

        print("Welcome to Hotel Chavishta\n")

        self.bill_id = datetime.now().strftime("%Y%m%d%H%M%S")

        self.show_menu()
        self.take_order()

        delivered = input("Is Order Delivered? (y/n): ").lower()

        if delivered != 'y':
            print("Order Not Delivered. Bill Cannot Be Generated")
            return

        generate = input("Generate Bill? (y/n): ").lower()

        if generate == 'y':
            total = self.generate_bill()
            if total > 0:
                self.save_bill()
        else:
            print("Bill Generation Cancelled")


# ----------------------------
# Run Program
# ----------------------------
system = RestaurantBilling()
system.run()


<>:11: SyntaxWarning: invalid escape sequence '\S'
<>:11: SyntaxWarning: invalid escape sequence '\S'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14804\2328791431.py:11: SyntaxWarning: invalid escape sequence '\S'
  "Server=DESKTOP-LJ2060M\SQLEXPRESS;"


Database Connected Successfully

Welcome to Hotel Chavishta


------ MENU ------
1 coffee ₹ 50
2 chai ₹ 20
3 cold coffee ₹ 70
4 fries ₹ 100
5 pizza ₹ 150
6 burger ₹ 90
7 pasta ₹ 180
8 momos ₹ 80
9 waffle ₹ 160
10 ice cream ₹ 70
------------------

Start Taking Order (Enter 0 to Stop)

Added: coffee x 1 

Added: pizza x 1 

Added: burger x 2 

Added: pasta x 2 

Added: waffle x 1 

Order Taking Stopped

--------------------------------------------------
|            Welcome Hotel Chavishta             |
--------------------------------------------------
| sr  Menu            qunt        price |
--------------------------------------------------
| 1   coffee          1              50 |
| 2   pizza           1             150 |
| 3   burger          2             180 |
| 4   pasta           2             360 |
| 5   waffle          1             160 |
--------------------------------------------------
| Total                             900 |
---------------------------------------------